# Extra Credit Assignment

### Noah Dunn

### 12/07/2022

On my honor as a student at the University of Virginia, I have neither given nor received unathorized aid on this assignment  -  *Noah Dunn*

In [1]:
%matplotlib inline
import theano
import pymc as pm
import sklearn
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#import wikipedia
import nltk
from nltk.corpus import stopwords
import json
import pandas as pd
import os
import scipy.stats as st
import pymc as pm
import scipy.linalg as la
import arviz as az
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
sns.set_style('white')
os.chdir('/Users/noahdunn/Desktop/MSDS/DS6040/Homeworks/ExtraCredit')

/Users/noahdunn/opt/anaconda3/lib/python3.9/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Question 1

With the BART Example code use BART to predict SAT scores using
the percent takes and school spending as predictors. Plot the 95% HDI for
posterior SAT scores vs. percent takers and comment on the relationship.

In [2]:
from pathlib import Path

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pymc_bart as pmb

az.style.use("arviz-darkgrid")
print(f"Running on PyMC v{pm.__version__}")

RANDOM_SEED = 1934
rng = np.random.RandomState(RANDOM_SEED)

df = pd.read_csv('/Users/noahdunn/Desktop/MSDS/DS6040/Homeworks/ExtraCredit/Guber1999data.csv')
df.info()

Running on PyMC v4.2.2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      50 non-null     object 
 1   Spend      50 non-null     float64
 2   StuTeaRat  50 non-null     float64
 3   Salary     50 non-null     float64
 4   PrcntTake  50 non-null     int64  
 5   SATV       50 non-null     int64  
 6   SATM       50 non-null     int64  
 7   SATT       50 non-null     int64  
dtypes: float64(3), int64(4), object(1)
memory usage: 3.2+ KB


In [3]:
X = df[['Spend', 'PrcntTake']]
Y = df['SATT']

meanx = X.mean().values
scalex = X.std().values
zX = ((X-meanx)/scalex).values

meanY = Y.mean()
scaleY = Y.std()
zY = ((Y-meanY)/scaleY).values

In [ ]:
with pm.Model() as model_bikes:
    σ = pm.HalfNormal("σ", sigma = zY.std())
    μ = pmb.BART("μ", zX, zY, m=50)
    y = pm.Normal("y", μ, σ, observed=zY)
    idata_bikes = pm.sample(random_seed=RANDOM_SEED)

Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>NUTS: [σ]
>PGBART: [μ]
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumP

In [ ]:
# SAT vs. Takers

_, ax = plt.subplots(figsize=(10, 6))

sat = idata_bikes.posterior["μ"]
az.plot_hdi(X['PrcntTake'], sat, smooth=True)
az.plot_hdi(X['PrcntTake'], sat, hdi_prob=0.5, smooth=True, plot_kwargs={"alpha": 0})
az.plot_hdi(X['PrcntTake'], sat, hdi_prob=0.1, smooth=True, plot_kwargs={"alpha": 0})
ax.set_xlabel("% of Students Taking SAT")
ax.set_ylabel("SAT Score");

#### Comments on relationship between SAT scores and % takers:  As the percent of students that take the SAT increases, the average score (generally) decreases. This is likely due to latent factors, such as the idea that in areas where there is a relatively small number of students taking the SAT, they are likely the only ones with a 'genuine chance' to go to a university where a good score might be required.

# Question 2

Explain the difference between traditional autoencoders and variational autoencoders. Give an example of how variational autoencoders
are used.

#### Difference:  A traditional autoencoder takes input data (image, vector, etc.) with high dimensionality and runs it through a neural network to compress the data into a smaller representation. The data then goes through a bottleneck where it is represented by the smallest number of variables, and then it is 'decoded' through another neural network that attempts to reconstruct the original data input. A variational autoencoder uses a similar neural network 'encoder' to compress data and a similar neural network 'decoder' to reconstruct the data. However, instead of mapping the input to a fixed vector (as would happen with a traditional encoder), it is mapped to a distribution. This means that two bottleneck vectors are used instead of one, with the first representing the mean of the distribution and the second representing the standard deviation of the distribution. This means that whenever you need to feed a vector through the decoder, the only thing that is needed is a sample from the distribution.

#### Variational Autoencoder Example:  One example is in reinforcement learning. The problem here is that there are very sparse rewards requiring a long time in order to train models. So, variational autoencoders are used as a sort of feature extractor, and then the goal is to be able to run another program/model/etc. on the compressed representation of the data produced by the variational autoencoder instead of on the entire (large) input space.

# Question 3

Neural networks are a graphical model, Markov Networks, that can be
analyzed with Bayesian methods and Boltzmann machines are examples.
Why is training or learning computationally challenging for Boltzmann
machines? Explain how restricted Boltzmann machines (RBM) are easier
to train. What approaches do we use for training RBM?

#### Why is training computationally challenging for Boltzmann machines:  In most real world problems, the number of cliques within the data being looked at is very large (and even if they are only of modest sizes). Because Boltzmann machines are made of nodes/neurons, and the connection matrix W describes whether neurons are connected with one another (can get very large), this makes the computation of $Z$ (the normalizing term) and $p(x_v, x_h)$ computationally intractable.

#### Why are restricted Boltzmann machines easier to train:  RBMs have a single hidden layer and a single visible layer. While the computation of $Z$ is intractable, the two layer structure of an RBM makes it feasible to calculate conditional distributions $P(x_h | x_v)$ and $P(x_v | x_h)$. The fact that there is no intra-layer communication between nodes (nodes within each of the two layers do not communicate with other nodes in that layer) allows for quicker training algorithms, such as contrastive divergence, to be used.

#### What approaches do we use to train RBMs:  RBMs are often trained using contrastive divergence, which uses Gibbs sampling inside a gradient descent procedure that computes updates for weights. This approximates the relationship between the error in the network and its weights, and then represents this as a slope/gradient in order to perform gradient descent.

# Question 4

The following questions are based on reading and running the jupyter
notebook, pymc3-variation-inference-neural-network.ipynb,
by Thomas Wiecki, updated by Maxim Kochurov as provided in their blog
post. Run the notebook and then answer these questions. Notice that this
notebook is in pymc3 not pymc.

### a)

Wieki says that an advantage to using Bayesian modeling with neural
networks and deep learning is that “we could train the model specifically on samples it is most uncertain about.” Explain how he finds
these samples in this example. Explain how you would implement
his suggestion (you do not have to actually implement this).

#### How he finds the samples with the most uncertainty in this example:  By training the model primarily on samples from the high-uncertainty region found when the standard deviation of the posterior predictive is graphed.

#### How could I implement his suggestion:  One way to do this would be through the use of Support Vector Machines that focus on values close to the decision boundary between the two 'clusters' in the model. SVMs only use support vectors that are close to the hyperplane being used to divide the data points, so this approach would make sense in order to implement the above solution.

### b)

Wieki also says that another advantage to Bayesian modeling with
neural network and deep learning is that “We also get uncertainty
estimates of our weights which could inform us about the stability
of the learned representations of the network.” Discuss what the
uncertainty estimates for the weights found for the example in this
notebook imply.

#### What the uncertainty estimates for the weights found for the example in this notebook imply:  In general, these uncertainty estimates tell us about how confident we are in the decision boundary being produced by the neural network. For example, if there is a high uncertainty/less stability for an area, it implies that bias could be present - if small changes to data were made around this area, the outcome could be very different. 

### c)

Explain how the Gaussian priors help to regularize the weights in the
neural network.

#### How Gaussian priors help regularize weights in the neural network:  In neural networks, if weights are large then this often means that the model is overfitting the training data that it was fed. In order to mitigate this issue, L2-regularization can be helpful for bringing the weights closer to 0. A Gaussian prior centered on 0 will act similarly, by producing a higher prior probability of the weights being close to 0, with tails of lower probability going out into areas where the weights might be very high.

### d)

Why do we use a variational approximation instead of sampling for
estimating the posterior of the weights?

#### Why use variational approximation instead of sampling for estimating the posterior of the weights:  We use variational approximation instead of sampling because it can scale better and will run faster, especially as neural networks get deeper and deeper with more layers.

### e)

Change the prior distributions for all three sets of the neural net
weights to Cauchy with location (alpha) = 0 and scale (beta) =
2. Rerun the remaining cells in the notebook and comment on any
changes you see from this.

#### What changes come from changing the prior distributions of neural net weights:  